# Live Demo: The NAFLD Proactive Health Engine

**Objective:** This notebook simulates the core AI functionalities of our platform. We will walk through a user's journey to demonstrate:
1.  **Real-Time Risk Forecasting:** Using our multi-modal deep learning model to predict a user's liver health biomarkers.
2.  **Intelligent Triage:** How our system decides the most critical health advice a user needs right now.
3.  **Generative AI Coaching:** Using a RAG architecture to provide personalized, empathetic, and actionable recommendations.

In [11]:
import requests
import json
import pandas as pd
import numpy as np

# --- Configuration ---
# These are the URLs where our two backend services are running
PREDICTION_API_URL = "http://localhost:5000/predict"
RECOMMENDATION_API_URL = "http://localhost:5001/recommend"

# Helper function to print JSON nicely
def print_json(data):
    print(json.dumps(data, indent=2))

print("Simulator is ready. All API endpoints are configured.")

Simulator is ready. All API endpoints are configured.


## Part 1: A Day in the Life of "Rohan"

Let's meet our user, Rohan. He's a 45-year-old developer from Bangalore. He's sedentary, has a family history of diabetes, and his health has been trending downwards.

It's 8 PM, and Rohan has just finished a large, high-calorie dinner. He logs this meal in our app.

In [12]:
# This is the data our backend compiles after Rohan logs his meal.

# 1. Today's static and nutritional data
rohan_user_data = {
    "age": 45, "gender": "M", "bmi": 29.5, "has_hereditary_risk": 1,
    "tee": 2400, "calorie_intake": 2950, "fat_grams": 120,
    "carbs_grams": 380, "protein_grams": 130, "energy_balance": 550,
    "cumulative_balance": 45000
}

# 2. The last 14 days of Rohan's watch data (his recent history)
rohan_watch_data = [
    {'daily_steps': 4120, 'active_minutes': 15, 'sleep_hours': 6.1, 'sleep_quality_score': 65, 'resting_heart_rate': 72, 'heart_rate_variability': 35},
    {'daily_steps': 3850, 'active_minutes': 11, 'sleep_hours': 5.8, 'sleep_quality_score': 62, 'resting_heart_rate': 73, 'heart_rate_variability': 34},
    {'daily_steps': 4300, 'active_minutes': 20, 'sleep_hours': 6.5, 'sleep_quality_score': 70, 'resting_heart_rate': 71, 'heart_rate_variability': 36},
    {'daily_steps': 3900, 'active_minutes': 12, 'sleep_hours': 6.2, 'sleep_quality_score': 66, 'resting_heart_rate': 72, 'heart_rate_variability': 35},
    {'daily_steps': 4050, 'active_minutes': 18, 'sleep_hours': 5.9, 'sleep_quality_score': 63, 'resting_heart_rate': 73, 'heart_rate_variability': 33},
    {'daily_steps': 3700, 'active_minutes': 10, 'sleep_hours': 6.0, 'sleep_quality_score': 64, 'resting_heart_rate': 72, 'heart_rate_variability': 34},
    {'daily_steps': 4500, 'active_minutes': 25, 'sleep_hours': 6.8, 'sleep_quality_score': 72, 'resting_heart_rate': 70, 'heart_rate_variability': 38},
    {'daily_steps': 4210, 'active_minutes': 19, 'sleep_hours': 6.3, 'sleep_quality_score': 68, 'resting_heart_rate': 71, 'heart_rate_variability': 36},
    {'daily_steps': 3990, 'active_minutes': 14, 'sleep_hours': 6.1, 'sleep_quality_score': 65, 'resting_heart_rate': 72, 'heart_rate_variability': 35},
    {'daily_steps': 4150, 'active_minutes': 16, 'sleep_hours': 6.4, 'sleep_quality_score': 69, 'resting_heart_rate': 71, 'heart_rate_variability': 37},
    {'daily_steps': 3800, 'active_minutes': 11, 'sleep_hours': 5.7, 'sleep_quality_score': 61, 'resting_heart_rate': 74, 'heart_rate_variability': 32},
    {'daily_steps': 4000, 'active_minutes': 15, 'sleep_hours': 6.0, 'sleep_quality_score': 64, 'resting_heart_rate': 72, 'heart_rate_variability': 35},
    {'daily_steps': 4350, 'active_minutes': 22, 'sleep_hours': 6.6, 'sleep_quality_score': 71, 'resting_heart_rate': 71, 'heart_rate_variability': 37},
    {'daily_steps': 4250, 'active_minutes': 20, 'sleep_hours': 6.2, 'sleep_quality_score': 67, 'resting_heart_rate': 72, 'heart_rate_variability': 36}
]

print("Rohan's data for today has been captured.")

Rohan's data for today has been captured.


## Part 2: The Prediction Engine

The moment Rohan logs his meal, our backend sends this data to our multi-modal deep learning model. The model analyzes his long-term habits (from the watch) and the immediate impact of his meal (from the log) to forecast his health risk.

In [13]:
# Construct the JSON payload for the prediction API
prediction_payload = {
    "user_data": rohan_user_data,
    "watch_data": rohan_watch_data
}

print("--- Sending data to the Prediction API... ---")
# Make the API call
response = requests.post(PREDICTION_API_URL, json=prediction_payload)
prediction_result = response.json()

print("\n--- Received Live Prediction from Model ---")
print_json(prediction_result)
rohan_predicted_tg = prediction_result.get('predicted_triglycerides', 210)

--- Sending data to the Prediction API... ---

--- Received Live Prediction from Model ---
{
  "predicted_ggt": 35.33000183105469,
  "predicted_triglycerides": 152.94000244140625
}


### Part 2b: Calculating the Fatty Liver Index (FLI)

The predicted TG and GGT values are powerful, but the **Fatty Liver Index (FLI)** combines these with BMI and Waist Circumference to provide a validated risk score for NAFLD. This score gives the user an immediate, understandable measure of their current liver health.

- **FLI < 30:** Low risk of steatosis (fatty liver).
- **FLI 30-60:** Intermediate risk.
- **FLI > 60:** High risk of steatosis.

In [15]:
import math

# --- Retrieve the necessary values ---
predicted_tg = prediction_result.get('predicted_triglycerides')
predicted_ggt = prediction_result.get('predicted_ggt')
# Get BMI from the persona data we defined earlier
user_bmi = rohan_user_data.get('bmi')

# The FLI formula also requires Waist Circumference. 
# For this simulation, we'll use a realistic value based on Rohan's high-risk profile.
waist_circumference_cm = 102 

print(f"--- Calculating FLI using the following inputs ---")
print(f"Predicted TG: {predicted_tg:.2f} mg/dL")
print(f"Predicted GGT: {predicted_ggt:.2f} U/L")
print(f"User BMI: {user_bmi:.2f}")
print(f"Assumed Waist Circumference: {waist_circumference_cm} cm\n")


# --- Calculate the FLI using the standard formula ---
# We use np.log for the natural logarithm (ln)
logit_p = (0.953 * np.log(predicted_tg) + 
           0.139 * user_bmi + 
           0.718 * np.log(predicted_ggt) + 
           0.053 * waist_circumference_cm - 15.745)

fli_score = (np.exp(logit_p) / (1 + np.exp(logit_p))) * 100


# --- Interpret the result ---
if fli_score < 30:
    risk_level = "LOW RISK"
    risk_color = "\033[92m" # Green
elif 30 <= fli_score < 60:
    risk_level = "INTERMEDIATE RISK"
    risk_color = "\033[93m" # Yellow
else:
    risk_level = "HIGH RISK"
    risk_color = "\033[91m" # Red

END_COLOR = "\033[0m"

print("--- Final Risk Assessment ---")
print(f"Calculated Fatty Liver Index (FLI): {risk_color}{fli_score:.2f}{END_COLOR}")
print(f"Interpretation: {risk_color}{risk_level} of Hepatic Steatosis{END_COLOR}")

--- Calculating FLI using the following inputs ---
Predicted TG: 152.94 mg/dL
Predicted GGT: 35.33 U/L
User BMI: 29.50
Assumed Waist Circumference: 102 cm

--- Final Risk Assessment ---
Calculated Fatty Liver Index (FLI): 75.30
Interpretation: HIGH RISK of Hepatic Steatosis


## Part 3: The AI Coach (Triage & RAG)

Our system sees the high Triglyceride prediction and that Rohan went over his calorie goal. The triage logic decides he needs **immediate dietary advice**.

It then calls our RAG-powered Recommendation Engine. It retrieves healthier alternatives from our nutrition database and provides all this context to the Gemini LLM to generate a personalized, empathetic response.

In [14]:
# The backend determines the recommendation type
recommendation_type = "DIET_FIRST"
user_context = {
    "predicted_tg": rohan_predicted_tg, # Use the live prediction!
    "calorie_target": 2400
}
# This is the meal Rohan just logged
original_meal = {
    "name": "Mutton Biryani",
    "calories": 850, "fat": 45, "protein": 40
}

# Construct the payload for the RAG API
rag_payload = {
    "recommendation_type": recommendation_type,
    "user_context": user_context,
    "original_meal": original_meal
}

print("--- Backend has triaged the situation. Requesting a DIET recommendation from the RAG Engine... ---")
# Make the API call
response = requests.post(RECOMMENDATION_API_URL, json=rag_payload)
recommendation_result = response.json()

print("\n--- Here is the AI Coach's personalized message to Rohan ---")
# Print the final recommendation text
print("\n" + recommendation_result.get('recommendation', "No recommendation received."))

--- Backend has triaged the situation. Requesting a DIET recommendation from the RAG Engine... ---

--- Here is the AI Coach's personalized message to Rohan ---

Hey there!  I saw you enjoyed some Mutton Biryani – it’s a delicious dish!  While it's okay to enjoy it occasionally, given your triglyceride levels and goal of preventing fatty liver disease, we need to focus on making some healthier choices moving forward.  Since we don't have a direct biryani replacement in our database, let's think strategically.  Your next meal could benefit from focusing on leaner proteins and more fiber.  We need to gradually reduce your fat intake.


Try to plan your next meal around something lighter.  While not a direct substitute for biryani,  we should aim to reduce the fat content considerably from your last meal.   Focusing on a balanced meal with plenty of vegetables will help manage your triglycerides.  Think about incorporating more plant-based proteins and whole grains in your overall diet as